In [1]:
import numpy as np
import os
from torch import nn
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from data_preparation import DataExtractor
from nn_training import train, set_random_seed

run_config = {
    'hidden_sizes': [100, 50, 50],
    'lr': 3e-5,
    'weight_decay': 0,
    'batch_size': 128
}

set_random_seed(57)



In [2]:
data_dir = '../Data/'

extractor = DataExtractor()

for base_element in ('Ti', 'Zr'):
    files_dir = os.path.join(data_dir, base_element)
    for file in os.listdir(files_dir):
        if file.endswith('.dat'):
            extractor.read_file(files_dir, file, 2, base_element)
        elif file.endswith('.unalloyed'):
            extractor.read_file(files_dir, file, 1, base_element)
files_dir = os.path.join(data_dir, 'Ternary')
for file in os.listdir(files_dir):
    if file.endswith('.dat'):
        extractor.read_file(files_dir, file, num_elements=3, base_element='Ti')

In [3]:
data = extractor.dataframe

In [4]:
features = data.apply(extractor.extract_properties, axis=1, result_type='expand').to_numpy()
target = data['a'].to_numpy()

In [5]:
def train_mlp(features_train, features_test, target_train, target_test):
    scaler = StandardScaler().fit(features_train)
    scaled_features_train = scaler.transform(features_train)
    scaled_features_test = scaler.transform(features_test)
    target_scaler = StandardScaler().fit(target_train.reshape(-1, 1))
    scaled_target_train = target_scaler.transform(target_train.reshape(-1, 1))
    scaled_target_test = target_scaler.transform(target_test.reshape(-1, 1))

    train_dataset = TensorDataset(torch.Tensor(scaled_features_train), torch.Tensor(scaled_target_train))
    test_dataset = TensorDataset(torch.Tensor(scaled_features_test), torch.Tensor(scaled_target_test))
    train_loader = DataLoader(train_dataset, batch_size=run_config['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=run_config['batch_size'], shuffle=False)

    model = nn.Sequential(nn.Linear(5, run_config['hidden_sizes'][0]),
                          nn.BatchNorm1d(run_config['hidden_sizes'][0]),
                          nn.ReLU(),
                          nn.Linear(run_config['hidden_sizes'][0], run_config['hidden_sizes'][1]),
                          nn.BatchNorm1d(run_config['hidden_sizes'][1]),
                          nn.ReLU(),
                          nn.Linear(run_config['hidden_sizes'][1], run_config['hidden_sizes'][2]),
                          nn.BatchNorm1d(run_config['hidden_sizes'][2]),
                          nn.ReLU(),
                          nn.Linear(run_config['hidden_sizes'][2], 1))

    optimizer = torch.optim.Adam(model.parameters(), lr=run_config['lr'], weight_decay=run_config['weight_decay'])
    loss = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=300, verbose=True)
    res_train, res_test = train(model, train_loader, test_loader, test_loader, loss, optimizer, "cuda:0",
                                n_epochs=3000, scheduler=scheduler, verbose=False,
                                check_dir=None, save_every=5,
                                model_name='nn_a', show_tqdm=False)
    return target_scaler.inverse_transform(np.array(res_train)), target_scaler.inverse_transform(np.array(res_test))

In [6]:
cv_mse_train = []
cv_mse_test = []
cv_r2_train = []
cv_r2_test = []

for train_index, test_index in KFold(n_splits=5, shuffle=True, random_state=57).split(features):
    features_train, features_test = features[train_index], features[test_index]
    target_train, target_test = target[train_index], target[test_index]
    res_train, res_test = train_mlp(features_train, features_test, target_train, target_test)

    cv_mse_train.append(mean_squared_error(target_train, res_train))
    cv_mse_test.append(mean_squared_error(target_test, res_test))
    cv_r2_train.append(r2_score(target_train, res_train))
    cv_r2_test.append(r2_score(target_test, res_test))

print(f"A MSE Train: {np.mean(cv_mse_train):.3f}±{np.std(cv_mse_train):.3f}\n"
      f"A MSE Test:  {np.mean(cv_mse_test):.3f}±{np.std(cv_mse_test):.3f}")
print(f"A R^2 Train: {np.mean(cv_r2_train):.3f}±{np.std(cv_r2_train):.3f}\n"
      f"A R^2 Test:  {np.mean(cv_r2_test):.3f}±{np.std(cv_r2_test):.3f}")

Epoch 01385: reducing learning rate of group 0 to 3.0000e-06.
Epoch 01686: reducing learning rate of group 0 to 3.0000e-07.
Epoch 01987: reducing learning rate of group 0 to 3.0000e-08.
Epoch 02288: reducing learning rate of group 0 to 3.0000e-09.
A MSE Train: 0.038±0.002
A MSE Test:  0.000±0.000
A R^2 Train: -0.971±0.056
A R^2 Test:  0.983±0.001


In [7]:
features = data.apply(extractor.extract_properties, axis=1, result_type='expand').to_numpy()
target = data['e'].to_numpy()

cv_mse_train = []
cv_mse_test = []
cv_r2_train = []
cv_r2_test = []

for train_index, test_index in KFold(n_splits=5, shuffle=True, random_state=57).split(features):
    features_train, features_test = features[train_index], features[test_index]
    target_train, target_test = target[train_index], target[test_index]
    res_train, res_test = train_mlp(features_train, features_test, target_train, target_test)

    cv_mse_train.append(mean_squared_error(target_train, res_train))
    cv_mse_test.append(mean_squared_error(target_test, res_test))
    cv_r2_train.append(r2_score(target_train, res_train))
    cv_r2_test.append(r2_score(target_test, res_test))

print(f"E MSE Train: {np.mean(cv_mse_train):.3f}±{np.std(cv_mse_train):.3f}\n"
      f"E MSE Test:  {np.mean(cv_mse_test):.3f}±{np.std(cv_mse_test):.3f}")
print(f"E R^2 Train: {np.mean(cv_r2_train):.3f}±{np.std(cv_r2_train):.3f}\n"
      f"E R^2 Test:  {np.mean(cv_r2_test):.3f}±{np.std(cv_r2_test):.3f}")

Epoch 01826: reducing learning rate of group 0 to 3.0000e-06.
Epoch 02127: reducing learning rate of group 0 to 3.0000e-07.
Epoch 02724: reducing learning rate of group 0 to 3.0000e-08.
Epoch 02532: reducing learning rate of group 0 to 3.0000e-06.
Epoch 02833: reducing learning rate of group 0 to 3.0000e-07.
Epoch 01309: reducing learning rate of group 0 to 3.0000e-06.
Epoch 01952: reducing learning rate of group 0 to 3.0000e-07.
Epoch 02253: reducing learning rate of group 0 to 3.0000e-08.
E MSE Train: 2158.031±151.071
E MSE Test:  200.305±97.366
E R^2 Train: -0.865±0.053
E R^2 Test:  0.827±0.062
